# Dados Numéricos — Kaggle Cardiovascular Disease

Este notebook instala a API do Kaggle, autentica, baixa o dataset `sulianova/cardiovascular-disease-dataset`, carrega com pandas e realiza uma EDA básica (head, info, describe).

In [ ]:
# Instalações básicas (Colab/Jupyter)
%pip -q install kaggle pandas
print('Dependências OK')

OSError: Could not find kaggle.json. Make sure it's located in C:\Users\JuniorRodriguesdaSil\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [ ]:
# A autenticação do Kaggle será tratada automaticamente na próxima célula se variáveis de ambiente existirem.
print('Se KAGGLE_USERNAME/KAGGLE_KEY estiverem definidos, o download usará essas credenciais.')

In [ ]:
# Download temporário e carregamento em DataFrame
import os, json, pathlib, sys, tempfile, subprocess, glob
import pandas as pd

_tmp = tempfile.TemporaryDirectory()
data_dir = pathlib.Path(_tmp.name) / 'numericos'
data_dir.mkdir(parents=True, exist_ok=True)

df = None
try:
    if os.getenv('KAGGLE_USERNAME') and os.getenv('KAGGLE_KEY'):
        # Garantir kaggle.json
        kdir = pathlib.Path.home() / '.kaggle'
        kdir.mkdir(exist_ok=True)
        cred_path = kdir / 'kaggle.json'
        cred_path.write_text(json.dumps({
            'username': os.getenv('KAGGLE_USERNAME'),
            'key': os.getenv('KAGGLE_KEY')
        }))
        # Baixar via Kaggle CLI
        cmd = [sys.executable, '-m', 'kaggle', 'datasets', 'download', '-d',
               'sulianova/cardiovascular-disease-dataset', '-p', str(data_dir), '--unzip']
        print('Baixando dataset Kaggle para:', data_dir)
        subprocess.check_call(cmd)
        files = glob.glob(str(data_dir / '*.csv'))
        if files:
            df = pd.read_csv(files[0])
            print('Arquivo carregado:', files[0])
        else:
            print('CSV não encontrado após download Kaggle. Usando amostra sintética...')
except Exception as e:
    print('Falha no download Kaggle:', e)

if df is None:
    # Fallback: amostra sintética mínima compatível
    import io
    csv_text = """id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,17500,2,169,62,120,80,1,1,0,0,1,0
1,20000,1,156,85,140,90,2,1,0,0,0,1
2,22000,2,165,70,160,100,3,2,1,1,1,1
3,19000,1,180,95,130,85,1,1,0,0,1,0
"""
    df = pd.read_csv(io.StringIO(csv_text))
    print('Usando amostra sintética (sem Kaggle).')

In [ ]:
# EDA básica
if 'df' not in globals() or df is None:
    raise RuntimeError('DataFrame não disponível. Verifique a célula de download/carregamento.')

display(df.head())
print(df.info())
display(df.describe(include='all'))